In [4]:
%pip install scrapy 
%pip install crochet 
%pip install scrapy-selenium

  Using cached Scrapy-2.11.2-py2.py3-none-any.whl.metadata (5.3 kB)
  Using cached cryptography-43.0.0-cp39-abi3-win_amd64.whl.metadata (5.4 kB)
  Using cached itemloaders-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached parsel-1.9.1-py2.py3-none-any.whl.metadata (11 kB)
  Using cached pyOpenSSL-24.2.1-py3-none-any.whl.metadata (13 kB)
  Using cached service_identity-24.1.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached tldextract-5.1.2-py3-none-any.whl.metadata (11 kB)
  Using cached cffi-1.17.0-cp312-cp312-win_amd64.whl.metadata (1.6 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached requests_file-2.1.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl.metadata (2.2 kB)
Using cached Scrapy-2.11.2-py2.py3-none-any.whl (290 kB)
Using cached cryptography-43.0.0-cp39-abi3-win_amd64.whl (3.1 MB)
Using cached itemloaders-1.3.1-py3-none-any.whl (12 kB)
Using cached parsel-1.9.1-py2.py3-none-any.whl (17 kB)
Usi

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python312\\Scripts\\tldextract.exe' -> 'c:\\Python312\\Scripts\\tldextract.exe.deleteme'


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached scrapy_selenium-0.0.7-py3-none-any.whl.metadata (3.9 kB)
  Using cached Scrapy-2.11.2-py2.py3-none-any.whl.metadata (5.3 kB)
  Using cached selenium-4.23.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached trio-0.26.2-py3-none-any.whl.metadata (8.6 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached scrapy_selenium-0.0.7-py3-none-any.whl (6.7 kB)
Using cached Scrapy-2.11.2-py2.py3-none-any.whl (290 kB)
Using cached selenium-4.23.1-py3-none-any.whl (9.4 MB)
Using cached trio-0.26.2-py3-none-any.whl (475 kB)
Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
Using cached PySocks-1.7.1-py3-none-any.whl (16 kB

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python312\\Scripts\\scrapy.exe' -> 'c:\\Python312\\Scripts\\scrapy.exe.deleteme'


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import scrapy 
from typing import Iterable
from scrapy_selenium import SeleniumRequest

class dataFormating(scrapy.item):

    card_name = scrapy.Field()

class databaseSpider(scrapy.Spider):
    name = "databaseSpider"

    custom_settings = {
        'FEED_FORMAT':'json',
        'FEED_URI':f'databaseSpider.json',
    }

def start_requests(self) -> Iterable[scrapy.Request]:
    base_url = 'https://gatherer.wizards.com/Pages/Search/Default.aspx?color=|[W]|[U]|[B]|[R]|[G]'

    for url in base_url:
        yield SeleniumRequest(url=url, callback=self.parse)
    return super().start_requests()


ModuleNotFoundError: No module named 'scrapy_selenium'

In [7]:
from scrapy.crawler import CrawlerProcess

process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})
    
process.crawl(databaseSpider)
process.start()

2024-08-19 12:45:31 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-08-19 12:45:31 [scrapy.utils.log] INFO: Versions: lxml 5.2.2.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.0, Twisted 24.7.0, Python 3.12.3 (tags/v3.12.3:f6650f9, Apr  9 2024, 14:05:25) [MSC v.1938 64 bit (AMD64)], pyOpenSSL 24.2.1 (OpenSSL 3.3.1 4 Jun 2024), cryptography 43.0.0, Platform Windows-10-10.0.19045-SP0


NameError: name 'databaseSpider' is not defined